<a href="https://colab.research.google.com/github/jan-kreischer/UZH_ML4NLP/blob/main/Project-06/ex06_tm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Project 6 - Topic Modeling 
using Latent Dirichlet Allocation (LDA)  
and Combined Topic Models (CTM).  
## 1. Setup
### 1.1 Dependencies
Installing all dependencies needed to run the simulations

In [1]:
!pip install contextualized-topic-models==2.2.0

     |████████████████████████████████| 78 kB 3.7 MB/s 
     |████████████████████████████████| 121 kB 17.2 MB/s 
     |████████████████████████████████| 366 kB 24.0 MB/s 
     |████████████████████████████████| 24.1 MB 1.2 MB/s 
     |████████████████████████████████| 785 kB 68.6 MB/s 
     |████████████████████████████████| 374 kB 67.7 MB/s 
     |████████████████████████████████| 3.3 MB 80.5 MB/s 
     |████████████████████████████████| 3.3 MB 70.7 MB/s 
     |████████████████████████████████| 1.2 MB 73.8 MB/s 
     |████████████████████████████████| 61 kB 678 kB/s 
     |████████████████████████████████| 596 kB 62.7 MB/s 
     |████████████████████████████████| 895 kB 89.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=384cde0ace74e5e370f92168310e0e20aa7edf9c3a929c92c366874f2356cbc5
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built s

### 1.2 Imports

In [2]:
import re
import random
import os
import urllib
import urllib.request
import gzip
import io
import csv
import random
from collections import defaultdict
from tqdm import tqdm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

### 1.3 Google Drive
We connect Google Drive in order to access stored data.

In [3]:
# Enable access to files stored in Google Drive
from google.colab import drive
# Leave this like it is
mountpoint = '/content/drive/' 
drive.mount(mountpoint)

Mounted at /content/drive/


In [4]:
# Adapt this path to the folder where your data is stored in google drive
base_path = 'My Drive/UZH_ML4NLP/Projects/Project-06/data' 
data_path = os.path.join(mountpoint, base_path)
# Cd into the directory with the git repo
% cd $data_path

[Errno 2] No such file or directory: '/content/drive/My Drive/UZH_ML4NLP/Projects/Project-06/data'
/content


### 1.4 Constants


In [5]:
NUM_LDA_TOPICS = 8 # The number of different topics to identify
NUM_FEATURES = 10000
MAX_DF=0.5
MIN_DF=0.01

In [6]:
# Path to the data files
path_before_1990 = 'titles_before_1990.txt'
path_from_1990_to_2009 = 'titles_from_1990_to_2009.txt'
path_from_2010 = 'titles_from_2010.txt'

### 1.5 Data Acquisition

In [7]:
# Execute the following cell only once to download the data and write it as a file to your google drive. Afterwards, skip this cell or comment it out.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# to download the data manually or get more information, go to: https://dblp.org/faq/How+can+I+download+the+whole+dblp+dataset.html
url = 'https://dblp.uni-trier.de/xml/dblp.xml.gz'
num_titles = 500000  # the (max)number of titles to load 


def load_gzip_file(url):
    """Download Gzip-file."""
    response = urllib.request.urlopen(url)
    compressed_file = io.BytesIO(response.read())
    decompressed_file = gzip.GzipFile(fileobj=compressed_file)
    return decompressed_file

def extract_titles(input_file, max_num=40000):
    """Extract title and publication year of dblp papers, given as input file.
    
    Divide the papers into 3 time periods. 
    
    Collect max max_num papers per time period.
    """
    pairs_before_1990 = []
    count_before_1990 = 0
    pairs_from_1990_to_2009 = []
    count_from_1990_to_2009 = 0
    pairs_from_2010 = []
    count_from_2010 = 0
    got_title = False
    for line in tqdm(input_file):
        line_str = line.decode('utf-8')
        if got_title: 
            # we have a title and check for the corresponding year
            year_result = re.search(r'<year>(.*)</year>', line_str)
            if year_result:
                # we also have the year and thus save the title-year pair
                year = int(year_result.group(1))
                if year < 1990:
                    pairs_before_1990.append((title, year))
                    count_before_1990 += 1
                elif year < 2010:
                    pairs_from_1990_to_2009.append((title, year))
                    count_from_1990_to_2009 += 1
                else:
                    pairs_from_2010.append((title, year))
                    count_from_2010 += 1
                got_title = False
        else:
            # we have no title and search for title
            result = re.search(r'<title>(.*)</title>', line_str)
            if result:
                title = result.group(1)
                if len(title.split(' ')) < 3:  
                    # only include titles with at least four words
                    continue
                got_title = True
        
        if count_before_1990 >= max_num and count_from_1990_to_2009 >= max_num and count_from_2010 >= max_num:
            return pairs_before_1990, pairs_from_1990_to_2009, pairs_from_2010
    
    return pairs_before_1990, pairs_from_1990_to_2009, pairs_from_2010

def save_data(pairs, file_path):
    with open(file_path, 'w') as fout:
        writer = csv.writer(fout)
        for pair in pairs:
            writer.writerow(pair)

in_file = load_gzip_file(url)
pairs_before_1990, pairs_from_1990_to_2009, pairs_from_2010 = extract_titles(in_file)
save_data(pairs_before_1990, path_before_1990)
save_data(pairs_from_1990_to_2009, path_from_1990_to_2009)
save_data(pairs_from_2010, path_from_2010)


Mounted at /content/drive


13531618it [00:38, 355850.56it/s]


## 2. Topic Modeling
### 2.1 Using Latent Dirichlet Allocation (LDA)

In [8]:
def load_titles(path):
  with open(path) as fin:
    reader = csv.reader(fin)
    titles = [row[0] for row in reader]
  return titles

In [9]:
# Simple text preprocessing by removing 
# all letters which are not in roman alphabet
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z ]', '', text)
    #text = re.sub(r'\b\w{1,3}\b', ' ', text)
    #text = re.sub(' +', ' ', text)
    text = text.lower()
    return text

In [10]:
# Now we turn the documents (or titles in this case) into a matrix feature representation.
def vectorize_data(titles, max_df=MAX_DF, min_df=MIN_DF, max_features=NUM_FEATURES):
  tf_vectorizer = CountVectorizer(max_df=max_df, min_df=min_df, max_features=max_features, stop_words='english')
  tf = tf_vectorizer.fit_transform(titles)
  tf_feature_names = tf_vectorizer.get_feature_names_out()
  return tf, tf_feature_names

#### 2.1.1 - Before the 1990s:

In [11]:
# Load the titles
titles_before_1990 = load_titles(path_before_1990)
print("{} titles before 1990".format(len(titles_before_1990)))

40000 titles before 1990


In [12]:
# Show some random samples
random.sample(titles_before_1990, 10)

['The Performance of Multicomputer Interconnection Networks.',
 'The Virtual Terminal Definition.',
 'Applications of the Drazin Inverse to the Hill Cryptographic System. Part III.',
 'Telset, the Finnish viewdata.',
 'Early History of the Association for Symbolic Logic.',
 'With the 849<sup>th</sup> SIS, 1942-45.',
 'Automatic Curve Fitting Using an Adaptive Local Algorithm.',
 'Strategic pattern generation: A solution technique for a class of games.',
 'Concurrency versus Availability: Atomic Mechanisms for Replicated Data.',
 'Inductive learning of relational productions.']

In [13]:
preprocessed_titles_before_1990 = [preprocess_text(title) for title in titles_before_1990]

In [14]:
# Show some preprocessed samples
random.sample(preprocessed_titles_before_1990, 10)

['a natural language shell',
 'clearing systems and isi isi inventory systems with nonlinear costs and positive lead times',
 'a cross decomposition algorithm for capacitated facility location',
 'nichtmonotonie im prototypischen dokumentkonfigurierungssystem kokon',
 'meeting of the association for symbolic logic santiago chile ',
 'a simple fortran support for computerassisted instruction',
 'an autolayout system for a hierarchical ic design environment',
 'a prewired storage unit',
 'report on the third ifac round table discussion session rt on fuzzy decision making and applications at the seventh ifac world congress',
 'patterns in program references']

In [15]:
tf_01, tf_feature_names_01 = vectorize_data(preprocessed_titles_before_1990, max_df=0.95, min_df=0.01)

In [16]:
lda_01 = LatentDirichletAllocation(n_components=8, max_iter=10, learning_method='online', random_state=42).fit(tf_01)

In [17]:
for topic_idx, topic in enumerate(lda_01.components_):
    print(f'Topic {topic_idx}:', end=' ')
    print(' '.join([tf_feature_names_01[i] for i in topic.argsort()[:-12 - 1:-1]]))

Topic 0: theory problems algorithms simulation decision parallel application solution applications optimal digital control
Topic 1: computer logic model programs digital performance design networks using applications systems simulation
Topic 2: problem programming optimal language digital processing software research solution parallel linear control
Topic 3: data method network models application languages solution processing problem using analysis programming
Topic 4: note information linear functions applications finite technical programming systems time problem decision
Topic 5: algorithm design analysis approach sets performance new using implementation parallel linear digital
Topic 6: systems using parallel performance implementation decision distributed linear control digital design processing
Topic 7: control networks new recognition distributed time pattern optimal systems approach digital linear


Topics:
0. Graph/networks algorithms (seems to be mostly about algorithms that (maybe) operate on graphs/networks)
1. pattern recognition (and maybe robotics)
2. ...

#### 2.1.2 - From 1990 to 2009:

In [18]:
titles_from_1990_to_2009 = load_titles(path_from_1990_to_2009)
print("{} titles from 1990 to 2009".format(len(titles_from_1990_to_2009)))

327307 titles from 1990 to 2009


In [19]:
random.sample(titles_from_1990_to_2009, 10)

['Modular Structural Operational Semantics with Strategies.',
 'The discrete wavelet transform and the scale analysis of the surface properties of sea ice.',
 "Unification of Legendre, Laguerre, Hermite, and binomial discrete transforms using Pascal's matrix.",
 'Nontraditional Careers for Chemists: New Formulas in Chemistry By Lisa M. Balbes. Oxford University Press, New York, NY, 2007. xi + 307 pp + bibliography and indexes. ISBN 0-19-518366-5, hardcover, U.S. $74.50. ISBN 0-19-518367-3, softcover, U.S. $27.95.',
 'An innovative real-time technique for buried object detection.',
 'A discontinuous Galerkin method for the Reissner-Mindlin plate in the primitive variables.',
 'Offering High Quality Reference Service on the Web: The Collaborative Digital Reference Service (CDRS).',
 'Community-Building with Web-Based Systems - Investigating a Hybrid Community of Students.',
 'The Dempster-Shafer calculus for statisticians.',
 'Web mining for Web image retrieval.']

In [20]:
preprocessed_titles_from_1990_to_2009 = [preprocess_text(title) for title in titles_from_1990_to_2009]

In [21]:
random.sample(preprocessed_titles_from_1990_to_2009, 10)

['storage systems for data warehousing',
 'characterizing the scale dimension of a highdimensional classification problem',
 'on a heleshawtype domain evolution with convected surface energy density',
 'programme pour le calcul de coefficients dassociation entre variables relationnelles',
 'environment mapped refraction models for low cost scanline rendering',
 'digital libraries and world wide web sites and page persistence',
 'outdoor recognition at a distance by fusing gait and face',
 'local asymptotic stability for nonlinear state feedback delay systems',
 'linear bspline copulas with applications to nonparametric estimation of copulas',
 'stability of systems with several monotone nonlinearities']

In [22]:
tf_02, tf_feature_names_02 = vectorize_data(titles_from_1990_to_2009, max_df=0.95, min_df=0.01)

In [23]:
lda_02 = LatentDirichletAllocation(n_components=8, max_iter=10, learning_method='online', random_state=42).fit(tf_02)

In [24]:
for topic_idx, topic in enumerate(lda_02.components_):
    print(f'Topic {topic_idx}:', end=' ')
    print(' '.join([tf_feature_names_02[i] for i in topic.argsort()[:-12 - 1:-1]]))

Topic 0: time algorithm linear new network models algorithms efficient high robust equations management
Topic 1: method study problems evaluation case space programming equations finite linear new performance
Topic 2: design approach nonlinear optimal fuzzy modeling computing robust control equations new time
Topic 3: based control model methods computer robust time linear simulation network detection dynamic
Topic 4: using analysis networks performance problem multi dynamic neural wireless mobile recognition network
Topic 5: systems data information multiple digital linear robust management time control nonlinear analysis
Topic 6: adaptive application structure non theory knowledge scheme management robust linear control finite
Topic 7: learning estimation applications order image distributed graphs web software power real development


#### 2.1.3 - From 2010 onwards:

In [25]:
# Load the titles
titles_from_2010 = load_titles(path_from_2010)
print("{} titles from from 2010".format(len(titles_from_2010)))

720322 titles from from 2010


In [26]:
# Show some random samples
random.sample(titles_from_2010, 10)

['Dynamic Available AGC Based Approach for Enhancing Utility Scale Energy Storage Performance.',
 'Learning Curves in Machine Learning.',
 'A Generalized Half-Wave Symmetry SHE-PWM Formulation for Multilevel Voltage Inverters.',
 'Counting conjugacy classes of elements of finite order in Lie groups.',
 'Local Sigmoid Method: Non-Iterative Deterministic Learning Algorithm for Automatic Model Construction of Neural Network.',
 'Distributed supervisory control of discrete-event systems with communication delay.',
 "Exploring managers' intention to use business intelligence: the role of motivations.",
 'A corner truncated fractal slot ultrawide spectrum sensing antenna for wireless cognitive radio sensor network.',
 'Investigating the Utility Potential of Low-Cost Unmanned Aerial Vehicles in the Temporal Monitoring of a Landfill.',
 'On Ensemble SSL Algorithms for Credit Scoring Problem.']

In [27]:
# Preprocess the titles by removing certain characters
preprocessed_titles_from_2010 = [preprocess_text(title) for title in titles_from_2010]

In [28]:
# Vectorize
tf_03, tf_feature_names_03 = vectorize_data(preprocessed_titles_from_2010, max_df=0.95, min_df=0.01)

In [29]:
lda_03 = LatentDirichletAllocation(n_components=8, max_iter=10, learning_method='online', random_state=42).fit(tf_03)

In [30]:
for topic_idx, topic in enumerate(lda_03.components_):
    print(f'Topic {topic_idx}:', end=' ')
    print(' '.join([tf_feature_names_03[i] for i in topic.argsort()[:-12 - 1:-1]]))

Topic 0: systems networks wireless performance novel evaluation sensing sensor communication linear distributed power
Topic 1: based detection design linear mobile recognition prediction computing images cloud feature method
Topic 2: data method network optimization application applications modeling equations methods hybrid cloud problems
Topic 3: adaptive nonlinear information models framework energy deep management social scheduling systems tracking
Topic 4: control model estimation robust sensor selection tracking systems linear nonlinear stochastic distributed
Topic 5: learning neural efficient optimal distributed scheme problem multiple problems stochastic machine deep
Topic 6: approach algorithm study new time algorithms online tracking case improved research optimization
Topic 7: using analysis image dynamic power classification fuzzy communication machine feature learning selection


# Combined Topic Models

New method developed by [Bianchi et al. 2021](https://aclanthology.org/2021.acl-short.96/). 

[A 6min presentation of the paper by one of the authors.](https://underline.io/lecture/25716-pre-training-is-a-hot-topic-contextualized-document-embeddings-improve-topic-coherence)

Code: [https://github.com/MilaNLProc/contextualized-topic-models](https://github.com/MilaNLProc/contextualized-topic-models)

Tutorial: [https://colab.research.google.com/drive/1fXJjr_rwqvpp1IdNQ4dxqN4Dp88cxO97?usp=sharing](https://colab.research.google.com/drive/1fXJjr_rwqvpp1IdNQ4dxqN4Dp88cxO97?usp=sharing)

Again, perform topic modelling for the three time periods - this time using the combined topic models (CTMs). 

You can use and adapt the code from the tutorial linked above.

Use the available GPU for faster running times.

In [31]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing

num_ctm_topics = 5  # you can also choose a higher number of topics

In [132]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def lemmatization(sentence):
  words = sentence.split()
  lem_words = [WordNetLemmatizer().lemmatize(i) for i in words]
  new_sentence = ' '.join(lem_words)
  return new_sentence

def load_text(text):
  lem_text = [lemmatization(i) for i in text]
  sp = WhiteSpacePreprocessing(lem_text, stopwords_language='english')
  preprocessed_documents, unpreprocessed_corpus, vocab = sp.preprocess()
  return preprocessed_documents, unpreprocessed_corpus, vocab

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Before the 1990s:

In [133]:
#check the processed data from part 1
preprocessed_titles_before_1990[0:5]

['object model capabilities for distributed object management',
 'distributed object management technology',
 'muffin a distributed database machine',
 'algebraical optimization of ftaexpressions',
 'wissensrepraumlsentation und maschinelles lernen']

In [134]:
# here we do lemmatization, remove stop words, and return the processed, unprocessed, and vocabulary data 
preprocessed_documents_before_1990, unpreprocessed_corpus_before_1990, vocab_before_1990 = load_text(preprocessed_titles_before_1990)

0it [01:48, ?it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [135]:
# here we use the same model, paraphrase-distilroberta-base-v1, as in the tutorial
tp = TopicModelDataPreparation("paraphrase-MiniLM-L12-v2")
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus_before_1990, text_for_bow=preprocessed_documents_before_1990)

Batches:   0%|          | 0/198 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [136]:
ctm1 = CombinedTM(bow_size=len(tp.vocab), contextual_size=384, n_components=50, num_epochs=20)
ctm1.fit(training_dataset) # run the model

Epoch: [20/20]	 Seen Samples: [791400/791400]	Train Loss: 48.38527203324409	Time: 0:00:05.729325: : 20it [01:57,  5.87s/it]


In [137]:
for i in range(0,8):
    print(f'Topic {i}:', end=' ')
    print(' '.join(ctm1.get_topic_lists(12)[i]))

Topic 0: function boolean equation algebra switching method recursive linear polynomial synthesis algebraic differential
Topic 1: grammar implication sentence contextfree axiom attribute semantics semantic statement definition parsing substitution
Topic 2: set property number relation finite group type degree recursive field algebra space
Topic 3: introduction report science engineering operation technology research simulation society computer software education
Topic 4: time process machine queue scheduling random production stochastic single distribution sequential cost
Topic 5: algorithm parallel fast efficient search binary polygon convex finding matrix hull tree
Topic 6: number function complexity finite automaton sequence boolean class set property cellular binary
Topic 7: editor letter review book new introduction intelligence artificial paper comment ai report


In [139]:
# compare with the result of LDA from part 1
for topic_idx, topic in enumerate(lda_01.components_):
    print(f'Topic {topic_idx}:', end=' ')
    print(' '.join([tf_feature_names_01[i] for i in topic.argsort()[:-12 - 1:-1]]))

Topic 0: theory problems algorithms simulation decision parallel application solution applications optimal digital control
Topic 1: computer logic model programs digital performance design networks using applications systems simulation
Topic 2: problem programming optimal language digital processing software research solution parallel linear control
Topic 3: data method network models application languages solution processing problem using analysis programming
Topic 4: note information linear functions applications finite technical programming systems time problem decision
Topic 5: algorithm design analysis approach sets performance new using implementation parallel linear digital
Topic 6: systems using parallel performance implementation decision distributed linear control digital design processing
Topic 7: control networks new recognition distributed time pattern optimal systems approach digital linear


### From 1990 to 2009

In [140]:
#check the processed data from part 1
preprocessed_titles_from_1990_to_2009[0:5]

['an evaluation of objectoriented dbms developments  edition',
 'darwin on the incremental migration of legacy information systems',
 'integrating heterogeneous autonomous distributed applications using the dom prototype',
 'integrating objectoriented applications and middleware with relational databases',
 'towards a transaction management system for dom']

In [141]:
# here we do lemmatization, remove stop words, and return the processed, unprocessed, and vocabulary data 
preprocessed_documents_from_1990_to_2009, unpreprocessed_corpus_from_1990_to_2009, vocab_from_1990_to_2009 = load_text(preprocessed_titles_from_1990_to_2009)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [142]:
tp = TopicModelDataPreparation("paraphrase-MiniLM-L12-v2")
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus_from_1990_to_2009, text_for_bow=preprocessed_documents_from_1990_to_2009)

Batches:   0%|          | 0/1621 [00:00<?, ?it/s]

Epoch: [18/20]	 Seen Samples: [12840948/14267720]	Train Loss: 57.99598536230298	Time: 0:01:39.765110: : 18it [38:18, 127.69s/it]
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [143]:
ctm2 = CombinedTM(bow_size=len(tp.vocab), contextual_size=384, n_components=50, num_epochs=20)
ctm2.fit(training_dataset) # run the model

Epoch: [20/20]	 Seen Samples: [6481000/6481000]	Train Loss: 53.10213343722275	Time: 0:00:46.052545: : 20it [15:16, 45.82s/it]


In [144]:
for i in range(0,8):
    print(f'Topic {i}:', end=' ')
    print(' '.join(ctm2.get_topic_lists(12)[i]))

Topic 0: channel signal estimation blind frequency error equalization interference noise spacetime ofdm offset
Topic 1: fast efficient algorithm parallel new computation implementation computing filter matrix transform adaptive
Topic 2: stability linear class delay nonlinear system stabilization robust feedback state timevarying discretetime
Topic 3: pattern processing recognition word language visual speech query object automatic temporal representation
Topic 4: distributed parallel realtime architecture memory processor computing processing implementation embedded multiprocessor efficient
Topic 5: case study use impact electronic information research technology empirical factor effect organizational
Topic 6: review digital isbn new library technology pp book future research science challenge
Topic 7: value method boundary solution problem solving numerical equation convergence elliptic element decomposition


### From 2010 onwards

In [145]:
#check the processed data from part 1
preprocessed_titles_from_2010[0:5]

['spectre attacks exploiting speculative execution',
 'computer science curricula ',
 'differences in productivity and impact across the different computer science subareas',
 'klaus tschira stiftung gemeinnuumltzige gmbh kts',
 'catchment classification by runoff behaviour with selforganizing maps som']

In [146]:
# here we do lemmatization, remove stop words, and return the processed, unprocessed, and vocabulary data 
preprocessed_documents_from_2010, unpreprocessed_corpus_from_2010, vocab_from_2010 = load_text(preprocessed_titles_from_2010)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [147]:
tp = TopicModelDataPreparation("paraphrase-MiniLM-L12-v2")
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus_from_2010, text_for_bow=preprocessed_documents_from_2010)

Batches:   0%|          | 0/3578 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [148]:
ctm3 = CombinedTM(bow_size=len(tp.vocab), contextual_size=384, n_components=50, num_epochs=20)
ctm3.fit(training_dataset) # run the model

Epoch: [20/20]	 Seen Samples: [14311840/14311840]	Train Loss: 59.71951828452425	Time: 0:01:40.232822: : 20it [33:32, 100.64s/it]


In [149]:
for i in range(0,8):
    print(f'Topic {i}:', end=' ')
    print(' '.join(ctm3.get_topic_lists(12)[i]))

Topic 0: parallel search algorithm optimization fast evolutionary particle genetic swarm improved colony multiobjective
Topic 1: space function code property construction sequence polynomial type two curve family set
Topic 2: case information health quality knowledge review electronic technology research record innovation impact
Topic 3: selection feature clustering novel hybrid based approach rule classification algorithm pattern support
Topic 4: based novel scheme encryption improved watermarking new adaptive efficient video chaotic fast
Topic 5: method new element finite solving order discontinuous numerical fractional equation domain boundary
Topic 6: virtual game student teaching reality course experience education development augmented use skill
Topic 7: fuzzy decision making set group theory linguistic measure decisionmaking intuitionistic operator new


### Summary 